In [ ]:
# %run "/content/drive/MyDrive/Colab Notebooks/spark-setup.ipynb"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q pycountry
import pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.6 MB/s eta 0:00:00


In [ ]:
# read countries
countries_df =spark.read.options(inferSchema=True, header=True).csv("/content/drive/MyDrive/datasets/countries/countries of the world.csv")
from pyspark.sql.functions import trim, ltrim, rtrim, col


In [ ]:
#Trim columns
countries_df = countries_df.withColumn("Country", rtrim(col("Country")))

In [ ]:
#Rename columns
countries_df = countries_df \
  .withColumnRenamed('Area (sq. mi.)','area_per_squaremile' ) \
  .withColumnRenamed('Pop. Density (per sq. mi.)','population_density_per_square_mile' ) \
  .withColumnRenamed('Coastline (coast/area ratio)','coastline_area_ratio' ) \
  .withColumnRenamed('Net migration','net_migration' ) \
  .withColumnRenamed('Infant mortality (per 1000 births)','infant_mortality_per_thousand' ) \
  .withColumnRenamed('GDP ($ per capita)', 'gdp') \
  .withColumnRenamed('Literacy (%)', 'literacy_rate') \
  .withColumnRenamed('Phones (per 1000)', 'phones_per_thousand') \
  .withColumnRenamed('Arable (%)', 'arable_percentage') \
  .withColumnRenamed('Crops (%)', 'crops_percentage') \
  .withColumnRenamed('Other (%)', 'other_percentage') \
  .withColumnRenamed('Climate', 'climate')


In [ ]:
countries_df.show()

+-----------------+--------------------+----------+-------------------+----------------------------------+--------------------+-------------+-----------------------------+-----+-------------+-------------------+-----------------+----------------+----------------+-------+---------+---------+-----------+--------+-------+
|          Country|              Region|Population|area_per_squaremile|population_density_per_square_mile|coastline_area_ratio|net_migration|infant_mortality_per_thousand|  gdp|literacy_rate|phones_per_thousand|arable_percentage|crops_percentage|other_percentage|climate|Birthrate|Deathrate|Agriculture|Industry|Service|
+-----------------+--------------------+----------+-------------------+----------------------------------+--------------------+-------------+-----------------------------+-----+-------------+-------------------+-----------------+----------------+----------------+-------+---------+---------+-----------+--------+-------+
|      Afghanistan|ASIA (EX. NEAR EAS

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def get_country_code(country_name):
    code = ""
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except:
        return None

convertCountry = udf(lambda z:get_country_code(z),StringType())


In [ ]:
get_country_code('Austria')

'AUT'

In [ ]:
countries_df = countries_df.withColumn("Country_Code", convertCountry(countries_df.Country))

In [ ]:
countries_df.show()

+-----------------+--------------------+----------+-------------------+----------------------------------+--------------------+-------------+-----------------------------+-----+-------------+-------------------+-----------------+----------------+----------------+-------+---------+---------+-----------+--------+-------+------------+
|          Country|              Region|Population|area_per_squaremile|population_density_per_square_mile|coastline_area_ratio|net_migration|infant_mortality_per_thousand|  gdp|literacy_rate|phones_per_thousand|arable_percentage|crops_percentage|other_percentage|climate|Birthrate|Deathrate|Agriculture|Industry|Service|Country_Code|
+-----------------+--------------------+----------+-------------------+----------------------------------+--------------------+-------------+-----------------------------+-----+-------------+-------------------+-----------------+----------------+----------------+-------+---------+---------+-----------+--------+-------+------------

In [ ]:
spark.sql("use olympics_db")

AnalysisException: [SCHEMA_NOT_FOUND] The schema `olympics_db` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [ ]:
countries_df.write.mode("overwrite").format("delta").saveAsTable("countries")

NameError: name 'countries_df' is not defined